In [1]:
import numpy as np
import pandas as pd
import collections
import string
import matplotlib.pyplot as plt
#from utils.features import FeatureExtractor
from utils.DatasetPrep import GatherImageSet
from skimage import io

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
image_dir = 'sample_dataset/images/'
truth_file = 'sample_dataset/ISIC-2017_Training_Part3_GroundTruth.csv'

num_images, image_names = GatherImageSet(image_dir)
images = []
image_ids = []
max_H, max_W = 0,0
for i in range(num_images):
    img = io.imread(image_dir+image_names[i])
    H, W, C = img.shape
    images.append(img)
    image_ids.append(image_names[i])
    max_H = max(H, max_H)
    max_W = max(W, max_W)
    if i == 100: break
print max_H, max_W

1944 2592


In [3]:
df = pd.DataFrame(data = {'image_id': image_ids, 'image': images})

truth = pd.read_csv(truth_file)

df = df.join(truth, rsuffix = '_other')

In [4]:
padded = np.zeros((len(images), max_H, max_W, 3))
for i in range(len(df['image'])):
    H, W, C = df['image'][i].shape
    padded[i, :H, :W, :C] = i

In [5]:
labels = df['melanoma']

In [6]:
training_examples = int(len(labels) * 0.75)

X_train = padded[:training_examples]
Y_train = labels[:training_examples]
X_test = padded[training_examples:]
Y_test = labels[training_examples:]

In [7]:
# dimensions of our images.
img_width, img_height = max_W, max_H

epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_height, img_width)
else:
    input_shape = (img_height, img_width, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, Y_train, validation_split=0.2, batch_size=batch_size, epochs=epochs)

model.save_weights('first_try.h5')

Train on 60 samples, validate on 15 samples
Epoch 1/50


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1051, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1011, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/Librar

IndexError: string index out of range